# Simple Decoder, Version 2

From Karpathy's [Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=6096s), at time 58:29.

#### NOTES

1. refactored model class constructor; `vocab_size` is defined globally, don't need to pass it around
1. added hyperparameter `n_embed` for number of channels C (size of embeddings)
1. added linear layer `lm_head` to create a spurious interaction (this will be developed out later)

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32   # how many independent sequences will we process in parallel?
block_size = 8    # what is the maximum context length for prediction?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embed = 32
# --------------


with open("input.txt", "r", encoding="UTF-8") as fin:
    text = fin.read()

# vocabulary of individual chars seen in text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# character-level tokenization for the character-based transformer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]           # encoder: take a string, output a list of int
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of int, output a string

# train and test splits, 90:10
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
valid_data = data[n:]


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else valid_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "valid"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [3]:
%%time

# for reproducibility
torch.manual_seed(1337)


class SimpleDecoder(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # each token directly reads off the logits
        # for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
 
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of ints
        token_embeddings = self.token_embedding_table(idx)    # (B,T,C)
        logits = self.lm_head(token_embeddings)               # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            
            # focus only on the last time step (bigram)
            logits = logits[:, -1, :] # becomes (B, c)
            
            # apply softmax to get probabilitis
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


model = SimpleDecoder()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    # every once in a while, evaluate the loss on train and validation sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss is {losses['train']:.4f}, validation loss {losses['valid']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
print("==========================")

step 0: train loss is 4.3886, validation loss 4.3734
step 300: train loss is 2.5267, validation loss 2.5399
step 600: train loss is 2.4998, validation loss 2.5315
step 900: train loss is 2.4903, validation loss 2.5085
step 1200: train loss is 2.4967, validation loss 2.5128
step 1500: train loss is 2.4809, validation loss 2.5020
step 1800: train loss is 2.4858, validation loss 2.5149
step 2100: train loss is 2.4865, validation loss 2.5000
step 2400: train loss is 2.4882, validation loss 2.5127
step 2700: train loss is 2.5006, validation loss 2.5117



CExthantrid owindike on, ble

HAPen bube d e.
S:
Ond my d?
LUMuss ar hthar usqur, t. bar dilasoaten wice my.

Hastacom o mup
Yowhthetof isth ble mil; dilll,

W:

Yees, hein lat Hetidrovets, and Wh p.
Gore y jomes l lind me l.
MAshe cechiry ptupr aisspllwhy.
Hurinde n Boopetelaves
MPORIII od mothakleo Windo wh t eiibys woutit,

Hive cend iend t so mower; te

AN ad nterupt f s ar irist m:

Thin maleronth,
Mad
RD:

Whio myr f-bube!
KENobuisar